# Data Cleaning 2: Missing Values

### 1. Determine all the variable types and find the fraction of the missing values for each variable.

In [93]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [94]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('SELECT * FROM useducation', con=engine)

engine.dispose()

In [95]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [96]:
education_df.isnull().sum()*100/education_df.count()

PRIMARY_KEY                       0.000000
STATE                             0.000000
YEAR                              0.000000
ENROLL                           21.399512
TOTAL_REVENUE                    16.562500
FEDERAL_REVENUE                  16.562500
STATE_REVENUE                    16.562500
LOCAL_REVENUE                    16.562500
TOTAL_EXPENDITURE                16.562500
INSTRUCTION_EXPENDITURE          16.562500
SUPPORT_SERVICES_EXPENDITURE     16.562500
OTHER_EXPENDITURE                21.399512
CAPITAL_OUTLAY_EXPENDITURE       16.562500
GRADES_PK_G                      13.115997
GRADES_KG_G                       9.705882
GRADES_4_G                        9.625276
GRADES_8_G                        9.625276
GRADES_12_G                       9.625276
GRADES_1_8_G                      9.625276
GRADES_9_12_G                     9.625276
GRADES_ALL_G                     13.115997
AVG_MATH_4_SCORE                178.358209
AVG_MATH_8_SCORE                180.451128
AVG_READING

### 2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

In [97]:
# Given the variables, it seems we want to keep as much data as possible. Unless we are looking for a per student basis, then I'd throw out any row with no ENROLL value.

education_df.describe()

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.000000,1.229000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.229000e+03,...,1361.000000,1361.000000,1361.00000,1.361000e+03,1.361000e+03,1.319000e+03,536.000000,532.000000,533.000000,498.000000
mean,2004.433646,9.159308e+05,9.092082e+06,7.663723e+05,4.216553e+06,4.109157e+06,9.196681e+06,4.762966e+06,2.680331e+06,4.292046e+05,...,64538.927259,64271.057311,54268.92432,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132
std,7.393983,1.065280e+06,1.174519e+07,1.145242e+06,5.543072e+06,5.482971e+06,1.197813e+07,6.293004e+06,3.353349e+06,5.340693e+05,...,80293.387625,78910.545776,67889.27623,6.436384e+05,3.077056e+05,9.707031e+05,10.221511,10.159722,7.769616,6.803725
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,633.000000,437.000000,311.00000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102
25%,1998.000000,2.583140e+05,2.186305e+06,1.893540e+05,1.153097e+06,7.158345e+05,2.165404e+06,1.168032e+06,6.357900e+05,1.028310e+05,...,13739.000000,13342.000000,10919.00000,1.106260e+05,5.147100e+04,1.810650e+05,229.694352,272.761598,214.663401,259.533171
50%,2004.000000,6.483130e+05,5.079546e+06,4.033765e+05,2.537074e+06,2.055780e+06,5.234506e+06,2.657452e+06,1.525406e+06,2.715960e+05,...,43272.000000,43339.000000,36735.00000,3.457750e+05,1.642600e+05,5.503420e+05,237.238552,280.618803,220.416034,265.010912
75%,2011.000000,1.014528e+06,1.085985e+07,8.289660e+05,5.080939e+06,4.768680e+06,1.074519e+07,5.568028e+06,3.229651e+06,5.186000e+05,...,75481.000000,76566.000000,67460.00000,6.119050e+05,2.905020e+05,9.282755e+05,241.995486,285.347428,223.999337,268.190121
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,493415.000000,500143.000000,498403.00000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130


In [98]:
# Since for every column the mean is skewed significantly away from the median, we'll use median to fill null values.

filled_all_imputation_education_df = education_df.fillna(education_df.median())

print(filled_all_imputation_education_df.isnull().sum())
print(filled_all_imputation_education_df.describe())
print('We can see that there are no more nulls, and the medians stayed the same, while the means lowered, so that worked.')

PRIMARY_KEY                     0
STATE                           0
YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_PK_G                     0
GRADES_KG_G                     0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_1_8_G                    0
GRADES_9_12_G                   0
GRADES_ALL_G                    0
AVG_MATH_4_SCORE                0
AVG_MATH_8_SCORE                0
AVG_READING_4_SCORE             0
AVG_READING_8_SCORE             0
dtype: int64
              YEAR        ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  \
count  1492.000000  1.492000e+03   1.492000e+03     1.492000e+03   
m

### 3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using *only* the observations for that specific year.

In [99]:
filled_by_year_imputation_education_df = pd.DataFrame(education_df)

for year in education_df['YEAR']:
    filled_by_year_imputation_education_df.loc[filled_by_year_imputation_education_df['YEAR'] == year].fillna(education_df.loc[education_df['YEAR'] == year].median(), inplace=True)

print(filled_by_year_imputation_education_df.isnull().sum())
filled_by_year_imputation_education_df.describe()

PRIMARY_KEY                       0
STATE                             0
YEAR                              0
ENROLL                          263
TOTAL_REVENUE                   212
FEDERAL_REVENUE                 212
STATE_REVENUE                   212
LOCAL_REVENUE                   212
TOTAL_EXPENDITURE               212
INSTRUCTION_EXPENDITURE         212
SUPPORT_SERVICES_EXPENDITURE    212
OTHER_EXPENDITURE               263
CAPITAL_OUTLAY_EXPENDITURE      212
GRADES_PK_G                     173
GRADES_KG_G                     132
GRADES_4_G                      131
GRADES_8_G                      131
GRADES_12_G                     131
GRADES_1_8_G                    131
GRADES_9_12_G                   131
GRADES_ALL_G                    173
AVG_MATH_4_SCORE                956
AVG_MATH_8_SCORE                960
AVG_READING_4_SCORE             959
AVG_READING_8_SCORE             994
dtype: int64


,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.000000,1.229000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.229000e+03,...,1361.000000,1361.000000,1361.00000,1.361000e+03,1.361000e+03,1.319000e+03,536.000000,532.000000,533.000000,498.000000
mean,2004.433646,9.159308e+05,9.092082e+06,7.663723e+05,4.216553e+06,4.109157e+06,9.196681e+06,4.762966e+06,2.680331e+06,4.292046e+05,...,64538.927259,64271.057311,54268.92432,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132
std,7.393983,1.065280e+06,1.174519e+07,1.145242e+06,5.543072e+06,5.482971e+06,1.197813e+07,6.293004e+06,3.353349e+06,5.340693e+05,...,80293.387625,78910.545776,67889.27623,6.436384e+05,3.077056e+05,9.707031e+05,10.221511,10.159722,7.769616,6.803725
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,633.000000,437.000000,311.00000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102
25%,1998.000000,2.583140e+05,2.186305e+06,1.893540e+05,1.153097e+06,7.158345e+05,2.165404e+06,1.168032e+06,6.357900e+05,1.028310e+05,...,13739.000000,13342.000000,10919.00000,1.106260e+05,5.147100e+04,1.810650e+05,229.694352,272.761598,214.663401,259.533171
50%,2004.000000,6.483130e+05,5.079546e+06,4.033765e+05,2.537074e+06,2.055780e+06,5.234506e+06,2.657452e+06,1.525406e+06,2.715960e+05,...,43272.000000,43339.000000,36735.00000,3.457750e+05,1.642600e+05,5.503420e+05,237.238552,280.618803,220.416034,265.010912
75%,2011.000000,1.014528e+06,1.085985e+07,8.289660e+05,5.080939e+06,4.768680e+06,1.074519e+07,5.568028e+06,3.229651e+06,5.186000e+05,...,75481.000000,76566.000000,67460.00000,6.119050e+05,2.905020e+05,9.282755e+05,241.995486,285.347428,223.999337,268.190121
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,493415.000000,500143.000000,498403.00000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130


In [100]:
# filled_all_imputation_education_df = education_df.fillna(education_df.median())

filled_by_year_imputation_education_df = education_df

for col in filled_by_year_imputation_education_df.columns:
    if filled_by_year_imputation_education_df[col].isnull().sum() == 0:
        continue
    else:
        for year in filled_by_year_imputation_education_df['YEAR']:
            mask = (filled_by_year_imputation_education_df['YEAR'] == year) & (filled_by_year_imputation_education_df[col].isnull())
            filled_by_year_imputation_education_df.loc[mask] = filled_by_year_imputation_education_df[col].median() 

In [101]:
# We can see in this case that again there are no nulls, but now the overall medians did change a little, because

print(filled_by_year_imputation_education_df.isnull().sum())
filled_by_year_imputation_education_df.describe()

PRIMARY_KEY                     0
STATE                           0
YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_PK_G                     0
GRADES_KG_G                     0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_1_8_G                    0
GRADES_9_12_G                   0
GRADES_ALL_G                    0
AVG_MATH_4_SCORE                0
AVG_MATH_8_SCORE                0
AVG_READING_4_SCORE             0
AVG_READING_8_SCORE             0
dtype: int64


,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,...,1492.000000,1492.000000,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1492.000000,1492.000000,1492.000000,1492.000000
mean,115569.323704,3.728959e+05,2.978594e+06,3.724967e+05,1.436365e+06,1.399759e+06,3.008306e+06,1.611695e+06,9.597863e+05,2.459953e+05,...,134769.946358,134956.129334,132520.991264,2.743501e+05,1.931090e+05,3.585896e+05,115079.130202,115091.008740,115074.338619,115086.585251
std,246544.423603,7.109172e+05,8.078069e+06,7.958797e+05,3.725582e+06,3.738373e+06,8.214564e+06,4.287060e+06,2.300368e+06,4.045810e+05,...,243578.855017,243518.428462,243298.240896,4.655923e+05,2.984451e+05,6.741089e+05,246771.534471,246766.003423,246773.765971,246768.062966
min,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,0.000000e+00,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,...,244.691866,244.691866,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,187.134670,232.831510,181.626990,236.379102
25%,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,...,244.691866,244.691866,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,244.691866,244.691866,227.228260,244.691866
50%,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,2.446919e+02,...,244.691866,244.691866,244.691866,2.446919e+02,2.446919e+02,2.446919e+02,244.691866,244.691866,244.691866,244.691866
75%,2013.500000,6.483130e+05,1.479703e+06,6.483130e+05,7.660610e+05,6.483130e+05,1.538029e+06,7.722782e+05,6.483130e+05,5.319942e+05,...,77625.750000,79269.250000,72332.750000,6.252868e+05,3.160865e+05,6.483130e+05,245.391379,288.724572,244.691866,270.281298
max,648313.000000,6.307022e+06,7.395890e+07,9.990221e+06,4.233364e+07,3.494151e+07,7.476609e+07,4.195426e+07,2.169368e+07,3.759373e+06,...,648313.000000,648313.000000,648313.000000,3.929869e+06,2.011865e+06,5.926631e+06,648313.000000,648313.000000,648313.000000,648313.000000


### 4. This time, fill in the missing values using interpolation (extrapolation).

In [102]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('SELECT * FROM useducation', con=engine)

engine.dispose()

In [103]:
print(education_df.isnull().sum())
filled_interpolation_education_df = education_df.interpolate(method='nearest')

print(filled_interpolation_education_df.isnull().sum())
filled_interpolation_education_df.describe()

PRIMARY_KEY                       0
STATE                             0
YEAR                              0
ENROLL                          263
TOTAL_REVENUE                   212
FEDERAL_REVENUE                 212
STATE_REVENUE                   212
LOCAL_REVENUE                   212
TOTAL_EXPENDITURE               212
INSTRUCTION_EXPENDITURE         212
SUPPORT_SERVICES_EXPENDITURE    212
OTHER_EXPENDITURE               263
CAPITAL_OUTLAY_EXPENDITURE      212
GRADES_PK_G                     173
GRADES_KG_G                     132
GRADES_4_G                      131
GRADES_8_G                      131
GRADES_12_G                     131
GRADES_1_8_G                    131
GRADES_9_12_G                   131
GRADES_ALL_G                    173
AVG_MATH_4_SCORE                956
AVG_MATH_8_SCORE                960
AVG_READING_4_SCORE             959
AVG_READING_8_SCORE             994
dtype: int64
PRIMARY_KEY                       0
STATE                             0
YEAR           

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.000000,1.250000e+03,1.286000e+03,1.286000e+03,1.286000e+03,1.286000e+03,1.286000e+03,1.286000e+03,1.286000e+03,1.250000e+03,...,1447.000000,1447.000000,1447.000000,1.447000e+03,1.447000e+03,1.446000e+03,1492.000000,1492.000000,1492.000000,1491.000000
mean,2004.433646,9.084059e+05,9.084260e+06,7.661415e+05,4.214604e+06,4.103515e+06,9.187719e+06,4.758071e+06,2.678150e+06,4.252118e+05,...,61579.761576,61294.467865,51734.341396,4.953893e+05,2.355370e+05,7.592488e+05,232.763656,275.807462,218.998717,263.278010
std,7.393983,1.058859e+06,1.172182e+07,1.142968e+06,5.533337e+06,5.471561e+06,1.195398e+07,6.280230e+06,3.346697e+06,5.309374e+05,...,78958.850132,77652.145300,66791.572997,6.330687e+05,3.027433e+05,9.424423e+05,10.264919,10.115267,6.746932,5.780933
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,633.000000,437.000000,311.000000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102
25%,1998.000000,2.490735e+05,2.187606e+06,1.895488e+05,1.156122e+06,7.159095e+05,2.166505e+06,1.169423e+06,6.363895e+05,1.005785e+05,...,10882.500000,11122.000000,9915.000000,8.718650e+04,4.288150e+04,1.641932e+05,225.377592,268.312020,216.273356,258.777258
50%,2004.000000,6.499335e+05,5.085954e+06,4.043130e+05,2.538567e+06,2.064416e+06,5.244116e+06,2.658850e+06,1.526175e+06,2.688195e+05,...,40903.000000,40261.000000,34790.000000,3.276290e+05,1.555060e+05,5.189445e+05,232.861332,276.689980,221.129754,262.118986
75%,2011.000000,1.006367e+06,1.088210e+07,8.300340e+05,5.109714e+06,4.761837e+06,1.074618e+07,5.579660e+06,3.232484e+06,5.142138e+05,...,73792.000000,73672.000000,65900.000000,5.957965e+05,2.870570e+05,8.957032e+05,241.672327,285.231170,223.411510,267.776050
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,493415.000000,500143.000000,498403.000000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130


### 5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?